## ライブブラリのインポート

In [1]:
import argparse
import random
import gym
import logging
from tqdm import tqdm

# 必要に応じて追加のライブラリをインポート


## パースとMiniWoB環境でcount-shapeタスクを実行する関数

In [2]:
import gym
from tqdm import tqdm
import logging

# 必要な場合は、LLMAgentクラスをインポート
# from llm_agent import LLMAgent


def parse_opt():
    parser = argparse.ArgumentParser()
    parser.add_argument("--env", type=str, default="count-shape")
    parser.add_argument("--num-episodes", type=int, default=10)
    parser.add_argument("--llm", type=str, default="chatgpt")
    parser.add_argument("--erci", type=int, default=0)
    parser.add_argument("--step", type=int, default=-1)
    parser.add_argument("--irci", type=int, default=1)
    parser.add_argument("--sgrounding", action="store_true", default=False)
    parser.add_argument("--headless", action="store_true", default=True)

    return parser.parse_args()


def get_html_state(opt, states):
    # HTML状態の取得ロジック（ダミー）
    return states[0].html_body

def record_history(success_rate, num_episodes):
    """成功率をhistory.txtに記録する"""
    with open('history.txt', 'a') as file:
        file.write(f"After {num_episodes} episodes, success rate: {success_rate*100:.2f}%\n")

def miniwob_count_shape(opt):
    env = gym.make("MiniWoBEnv-v0", env_name=opt.env, headless=opt.headless)
    success = 0

    for _ in tqdm(range(opt.num_episodes)):
        # LLMAgentの初期化とタスク設定
        llm_agent = LLMAgent(
            opt.env,
            rci_plan_loop=opt.erci,
            rci_limit=opt.irci,
            llm=opt.llm,
            state_grounding=opt.sgrounding,
        )

        states = env.reset(seeds=[random.random()], record_screenshots=True)
        llm_agent.set_goal(states[0].utterance)
        html_state = get_html_state(opt, states)
        llm_agent.update_html_state(html_state)

        try:
            instruction = llm_agent.generate_action()
            miniwob_action = llm_agent.convert_to_miniwob_action(instruction)
            states, rewards, dones, _ = env.step([miniwob_action])
        except ValueError:
            rewards = [0]
            dones = [True]

        if rewards[0] > 0:
            success += 1

    # 全エピソード終了後の成功率の計算と記録
    success_rate = success / opt.num_episodes
    print(f"Final success rate: {success_rate*100:.2f}%")
    record_history(success_rate, opt.num_episodes)

# opt変数の定義（ダミー）
class Options:
    def __init__(self):
        self.env = "count-shape"
        self.num_episodes = 10  # エピソード数
        self.erci = 1
        self.irci = 1
        self.llm = "chatgpt"
        self.sgrounding = True
        self.headless = True

if __name__ == "__main__":
    opt = Options()
    miniwob_count_shape(opt)


NameNotFound: Environment MiniWoBEnv doesn't exist. 

##  LLMAgentや必要なヘルパー関数を定義

In [1]:
class LLMAgent:
    def __init__(self, env_name, clip_results, **kwargs):
        self.env_name = env_name
        self.clip_results = clip_results
        # その他の初期化処理

    def set_goal(self, utterance):
        # ゴールの設定
        pass

    def update_html_state(self, html_state):
        # HTML状態の更新
        pass

    def generate_action(self):
        # アクション生成ロジック
        return "click_button"  # 例

    def convert_to_miniwob_action(self, instruction):
        # MiniWoBアクションへの変換
        return {}

    def save_result(self, success):
        # 結果の保存
        with open("history.txt", "a") as f:
            f.write(f"Task: {self.env_name}, Success: {success}\n")

def load_clip_results(file_path):
    # CLIP結果の読み込み
    return {}

def get_html_state(opt, states):
    # HTML状態の取得
    return states[0].html_body
